In [ ]:
!pip install ragas
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/5

# **Load data**

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Load ground truth
gt_df = pd.read_csv("/content/drive/MyDrive/model_outputs/test.csv")
gt_df


,question,answer,context
0,"Saya menggunakan GPS ketika berkendara, kata t...",ULASAN LENGKAP Intisari : Penggunaan GPS dinil...,{3714: 'pasal 105 setiap orang yang mengguna...
1,"Beberapa waktu lalu ada kabar, sepasang kekasi...",ULASAN LENGKAP dari artikel dengan judul Huk...,"{2957: ""['(1) setiap orang dilarang memprod..."
2,Saya pernah melakukan transaksi di salah satu ...,ULASAN LENGKAP . Jerat Hukum Menolak Pembay...,"{51: 'Artinya, dengan adanya sistem pembayaran..."
3,PSBB yang diterapkan di Jakarta masih diwarnai...,ULASAN LENGKAP Pembatasan Sosial Berskala Bes...,"{3370: '(1) Selama pemberlakuan PSBB, penduduk..."
4,Saya dan teman saya sudah tertipu oleh seorang...,ULASAN LENGKAP dari artikel dengan judul “ Pa...,"{2086: ""[['(1) Mencoba melakukan kejahatan dip..."
...,...,...,...
107,Ada seorang hansip yang menegur sekelompok pem...,ULASAN LENGKAP dari artikel dengan judul Ken...,"{2105: ""['(1) tidak ada satu perbuatan pun yan..."
108,Di lingkungan perumahan tempat saya berdomisil...,ULASAN LENGKAP dari artikel dengan judul sam...,{1866: 'Yang dimaksud tanggul adalah tanggul p...
109,Apa itu yang disebut dengan pledoi? Bagaimana ...,ULASAN LENGKAP dari artikel dengan judul Tat...,"{3074: ""['(1) a. setelah pemeriksaan dinya..."
110,Istri saya bercerita bahwa dulu dia saat berpa...,ULASAN LENGKAP dari artikel dengan judul Jer...,{4: 'Undang-Undang ini mulai berlaku setelah 3...


In [ ]:
import ast
def parse_dict(s):
    s = ast.literal_eval(s)
    list_s = []
    for key, value in s.items():
        value = value.replace("[", "<")
        value = value.replace("]", ">")
        value = value.replace("'", "")
        list_s.append(value)
    return list_s

gt_df["context"] = gt_df["context"].apply(parse_dict)

In [ ]:
#@title Load inference result from finetuned models
result_df = pd.read_csv("/content/drive/MyDrive/model_outputs/inference_result.csv", index_col=0)
result_df

,question,answer,context,text,model_answer
0,"Saya menggunakan GPS ketika berkendara, kata t...",ULASAN LENGKAP Intisari : Penggunaan GPS dinil...,{3714: 'pasal 105 setiap orang yang mengguna...,Below is a question paired with a context to t...,"[""<|begin_of_text|>Below is a question paired ..."
1,"Beberapa waktu lalu ada kabar, sepasang kekasi...",ULASAN LENGKAP dari artikel dengan judul Huk...,"{2957: ""['(1) setiap orang dilarang memprod...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
2,Saya pernah melakukan transaksi di salah satu ...,ULASAN LENGKAP . Jerat Hukum Menolak Pembay...,"{51: 'Artinya, dengan adanya sistem pembayaran...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
3,PSBB yang diterapkan di Jakarta masih diwarnai...,ULASAN LENGKAP Pembatasan Sosial Berskala Bes...,"{3370: '(1) Selama pemberlakuan PSBB, penduduk...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
4,Saya dan teman saya sudah tertipu oleh seorang...,ULASAN LENGKAP dari artikel dengan judul “ Pa...,"{2086: ""[['(1) Mencoba melakukan kejahatan dip...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
...,...,...,...,...,...
107,Ada seorang hansip yang menegur sekelompok pem...,ULASAN LENGKAP dari artikel dengan judul Ken...,"{2105: ""['(1) tidak ada satu perbuatan pun yan...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
108,Di lingkungan perumahan tempat saya berdomisil...,ULASAN LENGKAP dari artikel dengan judul sam...,{1866: 'Yang dimaksud tanggul adalah tanggul p...,Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
109,Apa itu yang disebut dengan pledoi? Bagaimana ...,ULASAN LENGKAP dari artikel dengan judul Tat...,"{3074: ""['(1) a. setelah pemeriksaan dinya...",Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...
110,Istri saya bercerita bahwa dulu dia saat berpa...,ULASAN LENGKAP dari artikel dengan judul Jer...,{4: 'Undang-Undang ini mulai berlaku setelah 3...,Below is a question paired with a context to t...,['<|begin_of_text|>Below is a question paired ...


In [ ]:
def filter_answer(text):
    pattern = "### Answer:\\n<|end_of_text|>"
    begin = text.find(pattern)
    text = text[begin+len(pattern):]
    return text.strip()

result_df["model_answer"] = result_df["model_answer"].apply(filter_answer)

# **Load model**

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas import EvaluationDataset
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import os
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.25,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=60,  # Wake up every 5s to check whether allowed to make a request,
)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    timeout=None,
    rate_limiter=rate_limiter,
    # other params...
)

Enter your Google AI API key: ··········


In [ ]:
contexts = gt_df["context"].tolist()
sample_queries = gt_df["question"].tolist()
expected_responses = gt_df["answer"].tolist()
model_responses = result_df["model_answer"].tolist()

test_data = []
for query,reference,context,response in zip(sample_queries,expected_responses, contexts, model_responses):

    test_data.append(
        {
            "user_input":query,
            "retrieved_contexts":context,
            "response":response,
            "reference":reference
        }
    )

evaluation_dataset = EvaluationDataset.from_list(test_data)

In [ ]:
#@title # run evaluation
from ragas.metrics import Faithfulness, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(model)

result = evaluate(dataset=evaluation_dataset,metrics=[Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

ERROR:ragas.executor:Exception raised in Job[213]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[214]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[215]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[218]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[219]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[220]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[222]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[223]: TimeoutError()


{'faithfulness': 0.2365, 'factual_correctness(mode=f1)': 0.2513}

In [ ]:
import pickle

with open('/content/drive/MyDrive/model_outputs/evaluation_results.pkl', 'wb') as file:
    pickle.dump(result, file)

In [ ]:
# load pickle
import pickle
with open('/content/drive/MyDrive/model_outputs/evaluation_results.pkl', 'rb') as file:
    result = pickle.load(file)

In [ ]:
len(result.traces)

112